<a href="https://colab.research.google.com/github/Federaffo/spring/blob/main/springWithCuttedAmr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Federaffo/spring.git
%cd spring
# Install the requirements
# - The code only works with transformers < 3.0 because of a disrupting change in positional embeddings
# - The code works fine with torch 1.5
!pip install -r requirements.txt
%cp ../scripts/predict_amrs_from_plaintext.py bin/
!pip install -e .

# Preprocessing requirements
!pip install pandas
!pip install spacy
!pip install penman
!python3 -m spacy download en_core_web_sm

Cloning into 'spring'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 117 (delta 43), reused 44 (delta 8), pack-reused 0
Receiving objects: 100% (117/117), 621.73 KiB | 2.40 MiB/s, done.
Resolving deltas: 100% (43/43), done.
/content/spring
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 264 kB 59.4 MB/s 
     |████████████████████████████████| 116 kB 60.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 674 kB 61.8 MB/s 
     |████████████████████████████████| 1.9 MB 57.0 MB/s 
     |████████████████████████████████| 880 kB 63.3 MB/s 
     |████████████████████████████████| 5.6 MB 33.2 MB/s 
     |██████████████████

In [7]:
!python bin/train.py --config configs/config.yaml --direction amr 

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
{'name': 'baseline+smart_init', 'model': 'facebook/bart-large', 'penman_linearization': True, 'use_pointer_tokens': True, 'raw_graph': False, 'remove_wiki': False, 'dereify': False, 'collapse_name_ops': False, 'batch_size': 500, 'beam_size': 1, 'dropout': 0.25, 'attention_dropout': 0.0, 'smart_init': True, 'accum_steps': 10, 'warmup_steps': 1, 'training_steps': 250000, 'weight_decay': 0.004, 'grad_norm': 2.5, 'scheduler': 'constant', 'learning_rate': 5e-05, 'max_epochs': 30, 'save_checkpoints': True, 'log_wandb': False, 'warm_start': True, 'use_recategorization': False, 'best_loss': False, 'remove_longer_than': 1024, 'train': 'mydata/train/*.txt', 'dev': 'mydata/dev/*.txt', 'test': 'mydata/test/*.txt'}
AMRBartForConditionalGeneration(
  (model): AMRBartModel(
    (shared): Embedding(53587, 1024)
    (encoder): AMRBartEncoder(
      (embed_tokens): Embedding(53587, 1024)
      (embe

In [9]:
!python bin/predict_amrs.py \
    --datasets mydata/test/*.txt \
    --gold-path data/tmp/dev-gold.txt \
    --pred-path data/tmp/dev-pred.txt \
    --checkpoint runs/0/best-smatch_checkpoint_30_0.5874.pt \
    --beam-size 5 \
    --batch-size 500 \
    --device cuda \
    --penman-linearization --use-pointer-tokens

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
  0% 0/260 [00:00<?, ?it/s]/content/spring/spring_amr/modeling_bart.py:1102: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size
100% 260/260 [00:43<00:00,  5.95it/s]
Building failure:
['(', '<pointer:0>', 'cause-01', ':ARG0', '(', '<pointer:1>', 'amr-unknown', ')', ':ARG1', '<pointer:1>']
[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
list index out of range
Smatch: 0.589
